In [1]:
import ast
import codecs
import pandas as pd
import numpy as np

df = pd.read_csv('../data/trd_feated.csv',encoding='utf-8-sig')
df.head()

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,s1_ask,s2_ask,len_l1l2_tkd_avg,common_wc_nzd,s1_uq_wc_nzd,s2_uq_wc_nzd,bleu1,bleu2,bleu3,bleu4
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,True,False,10.0,0.300000,0.250000,0.687500,0.333333,0.283654,2.290650e-01,1.708192e-01
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,0.892857,...,False,False,7.0,0.142857,0.777778,1.000000,0.242612,0.127868,6.475245e-103,4.154123e-155
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,1.090909,...,False,False,7.5,0.400000,0.571429,0.625000,0.461538,0.277350,1.279447e-102,7.855727e-155
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,1.875000,...,True,False,4.0,0.500000,0.500000,0.500000,0.495359,0.463366,4.237359e-01,3.564026e-01
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,0.850000,...,False,False,4.5,0.222222,0.666667,0.833333,0.363636,0.190693,9.991803e-103,6.513869e-155


In [2]:
df.columns

Index(['s1', 's2', 'label', 'len_s1', 'len_s2', 'len_s1s2_avg', 'len_s1_on_s2',
       'len_diff', 'edit_dst', 'edit_dst_nzd', 'l1_tkd', 'l2_tkd',
       'len_l1_tkd', 'len_l2_tkd', 'common_wc', 's1_uq_wc', 's2_uq_wc',
       's1_ask', 's2_ask', 'len_l1l2_tkd_avg', 'common_wc_nzd', 's1_uq_wc_nzd',
       's2_uq_wc_nzd', 'bleu1', 'bleu2', 'bleu3', 'bleu4'],
      dtype='object')

In [3]:
def load_dense_drop_repeat(path):
    vocab_size, size = 0, 0
    vocab = dict()
    vocab["i2w"], vocab["w2i"] = [], {}
    count = 0
    with codecs.open(path, "r", "utf-8") as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                vocab_size = int(line.strip().split()[0])
                size = int(line.rstrip().split()[1])
                matrix = np.zeros(shape=(vocab_size, size), dtype=np.float32)
                continue
            vec = line.strip().split()
            if not vocab["w2i"].__contains__(vec[0]):
                vocab["w2i"][vec[0]] = count
                matrix[count, :] = np.array([float(x) for x in vec[1:]])
                count += 1
    for w, i in vocab["w2i"].items():
        vocab["i2w"].append(w)
    return matrix, vocab, size, len(vocab["i2w"])

matrix, vocab, size, length = load_dense_drop_repeat('../word2vec/sgns.zhihu.word')

In [4]:
matrix.shape # OHE->EMD

(259922, 300)

In [5]:
len(vocab['i2w'])  # list: [word1, word2 ...]

259836

In [6]:
len(vocab['w2i'])  # dict: {word: index}

259836

In [7]:
length

259836

In [8]:
def get_token_list_idx(row):

    l1_idx, l2_idx = [], []
    l1_no_oov, l2_no_oov = [], []
    l1_emd, l2_emd = [], []

    l1_tkd = ast.literal_eval(row['l1_tkd'])
    l2_tkd = ast.literal_eval(row['l2_tkd'])

    for word in l1_tkd:
        if word in vocab['w2i'].keys():
            l1_idx.append(vocab['w2i'][word])
            l1_no_oov.append(word)
            l1_emd.append(matrix[vocab['w2i'][word]])

    for word in l2_tkd:
        if word in vocab['w2i'].keys():
            l2_idx.append(vocab['w2i'][word])
            l2_no_oov.append(word)
            l2_emd.append(matrix[vocab['w2i'][word]])

    return l1_no_oov, l2_no_oov, l1_idx, l2_idx,  l1_emd, l2_emd

df[['l1_no_oov', 'l2_no_oov', 'l1_idx', 'l2_idx', 'l1_emd', 'l2_emd']]=df.apply(get_token_list_idx, axis=1,result_type='expand')

In [9]:
def join(x):
    return ' '.join(x)

df['l1_no_oov_str']=df['l1_no_oov'].apply(lambda x:join(x))
df['l2_no_oov_str']=df['l2_no_oov'].apply(lambda x:join(x))

In [10]:
df.head()

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,bleu3,bleu4,l1_no_oov,l2_no_oov,l1_idx,l2_idx,l1_emd,l2_emd,l1_no_oov_str,l2_no_oov_str
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,2.290650e-01,1.708192e-01,"[怎么, 更改, 手机号码]","[我, 的, 是, 以前, 的, 手机号码, 怎么, 更, 改成, 现在, 的, 支付宝, ...","[176, 11047, 28645]","[8, 1, 5, 445, 1, 28645, 176, 123, 3981, 111, ...","[[-0.100047, 0.514658, -0.443213, 0.046349, -0...","[[0.005664, 0.407404, -0.153532, 0.161316, 0.1...",怎么 更改 手机号码,我 的 是 以前 的 手机号码 怎么 更 改成 现在 的 支付宝 的 号码 手机号
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,0.892857,...,6.475245e-103,4.154123e-155,"[也, 开, 不了, 就, 这样, 了, 完事, 了]","[真的, 嘛, 就是, 付款]","[27, 242, 666, 42, 131, 3, 14073, 3]","[143, 688, 23, 12050]","[[-0.075654, 0.515432, 0.187096, -0.124749, -0...","[[-0.173466, 0.344304, -0.239642, -0.079505, -...",也 开 不了 就 这样 了 完事 了,真的 嘛 就是 付款
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,1.090909,...,1.279447e-102,7.855727e-155,"[冻结, 以后, 还, 能, 开通, 吗]","[我, 的, 条件, 可以, 开通, 借款, 吗]","[20083, 311, 36, 48, 9777, 148]","[8, 1, 891, 28, 9777, 15366, 148]","[[-0.000582, 0.319691, -0.037281, -0.070775, 0...","[[0.005664, 0.407404, -0.153532, 0.161316, 0.1...",冻结 以后 还 能 开通 吗,我 的 条件 可以 开通 借款 吗
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,1.875000,...,4.237359e-01,3.564026e-01,"[如何, 得知, 关闭]","[想, 永久, 关闭]","[267, 2818, 4529]","[87, 8120, 4529]","[[0.045856, 0.289823, -0.431393, -0.103468, -0...","[[0.187698, 0.021538, -0.287818, 0.01359, -0.0...",如何 得知 关闭,想 永久 关闭
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,0.850000,...,9.991803e-103,6.513869e-155,[付钱],"[二维码, 扫描, 可以, 用, 吗]",[14255],"[3019, 8676, 28, 63, 148]","[[0.078619, 0.378632, -0.392645, 0.104507, 0.2...","[[-0.352213, 0.434613, -0.796925, -0.061786, 0...",付钱,二维码 扫描 可以 用 吗


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf1 = TfidfVectorizer(lowercase=False,vocabulary=vocab['w2i'])
dmt_l1_no_oov = tf1.fit_transform(df['l1_no_oov_str'])

In [12]:
idx_matrix_l1 = df['l1_idx'].values
tfidf_matrix_l1 = []
for i,i_list in enumerate(idx_matrix_l1):
    temp = []
    for j in i_list:
        temp.append(dmt_l1_no_oov[i,j])
    tfidf_matrix_l1.append(temp)

tfidf_matrix_l1[:5]

[[0.2911125346008847, 0.6813844622784201, 0.6715420364830462],
 [0.0,
  0.0,
  0.3050283321451065,
  0.0,
  0.5673142076102956,
  0.0,
  0.7649263405271629,
  0.0],
 [0.618902920804913, 0.6824507805103563, 0.0, 0.0, 0.38887029559989894, 0.0],
 [0.3543126968453009, 0.8674971456062696, 0.3491578657558564],
 [1.0]]

In [13]:
tf2 = TfidfVectorizer(lowercase=False,vocabulary=vocab['w2i'])
dmt_l2_no_oov = tf2.fit_transform(df['l2_no_oov_str'])
idx_matrix_l2 = df['l2_idx'].values
tfidf_matrix_l2 = []

for i,i_list in enumerate(idx_matrix_l2):
    temp = []
    for j in i_list:
        temp.append(dmt_l2_no_oov[i,j])
    tfidf_matrix_l2.append(temp)

tfidf_matrix_l2[:5]  # why has 0?

[[0.0,
  0.0,
  0.0,
  0.3720269035410146,
  0.0,
  0.40295686488792987,
  0.17677958180509531,
  0.0,
  0.4911574419957298,
  0.28322364678747375,
  0.0,
  0.2615859438026379,
  0.0,
  0.390868923104766,
  0.3539968072830744],
 [0.7366695145856915, 0.0, 0.5597105462850577, 0.37952882717570235],
 [0.0,
  0.0,
  0.6532393435180363,
  0.30830554760116524,
  0.36506168060505184,
  0.5873295657059039,
  0.0],
 [0.0, 0.8543087394543378, 0.5197658873877163],
 [0.559491271304203, 0.7860995812236942, 0.26271080246982514, 0.0, 0.0]]

In [14]:
df['tfidf_l1'] = tfidf_matrix_l1
df['tfidf_l2'] = tfidf_matrix_l2

In [15]:
df.head(1)

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,l1_no_oov,l2_no_oov,l1_idx,l2_idx,l1_emd,l2_emd,l1_no_oov_str,l2_no_oov_str,tfidf_l1,tfidf_l2
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,"[怎么, 更改, 手机号码]","[我, 的, 是, 以前, 的, 手机号码, 怎么, 更, 改成, 现在, 的, 支付宝, ...","[176, 11047, 28645]","[8, 1, 5, 445, 1, 28645, 176, 123, 3981, 111, ...","[[-0.100047, 0.514658, -0.443213, 0.046349, -0...","[[0.005664, 0.407404, -0.153532, 0.161316, 0.1...",怎么 更改 手机号码,我 的 是 以前 的 手机号码 怎么 更 改成 现在 的 支付宝 的 号码 手机号,"[0.2911125346008847, 0.6813844622784201, 0.671...","[0.0, 0.0, 0.0, 0.3720269035410146, 0.0, 0.402..."


In [16]:
def get_sent_emd(row):

    tfidf_l1 = row['tfidf_l1']
    tfidf_l2 = row['tfidf_l2']

    l1_emd = row['l1_emd']
    l2_emd = row['l2_emd']

    sent_emd_l1 = np.zeros(300)
    sent_emd_l2 = np.zeros(300)

    for a,b in zip(l1_emd,tfidf_l1):
        sent_emd_l1 += np.array(a)*b

    for a,b in zip(l2_emd,tfidf_l2):
        sent_emd_l2 += np.array(a)*b

    return sent_emd_l1, sent_emd_l2

df[['sent_emd_l1', 'sent_emd_l2']] = df.apply(get_sent_emd, axis=1,result_type='expand')

In [17]:

df.head(1)

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,l1_idx,l2_idx,l1_emd,l2_emd,l1_no_oov_str,l2_no_oov_str,tfidf_l1,tfidf_l2,sent_emd_l1,sent_emd_l2
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,"[176, 11047, 28645]","[8, 1, 5, 445, 1, 28645, 176, 123, 3981, 111, ...","[[-0.100047, 0.514658, -0.443213, 0.046349, -0...","[[0.005664, 0.407404, -0.153532, 0.161316, 0.1...",怎么 更改 手机号码,我 的 是 以前 的 手机号码 怎么 更 改成 现在 的 支付宝 的 号码 手机号,"[0.2911125346008847, 0.6813844622784201, 0.671...","[0.0, 0.0, 0.0, 0.3720269035410146, 0.0, 0.402...","[-0.4842908885329962, 0.6463035643100739, -0.4...","[-0.5333490184275433, 1.1786049455404282, -0.6..."


In [18]:
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance

def get_distances(row):
    sent_emd_l1 =row['sent_emd_l1']
    sent_emd_l2 =row['sent_emd_l2']
    return pairwise_distances([sent_emd_l1],[sent_emd_l2],metric='euclidean')[0][0],\
           pairwise_distances([sent_emd_l1],[sent_emd_l2],metric='manhattan')[0][0],\
           pairwise_distances([sent_emd_l1],[sent_emd_l2],metric='cosine')[0][0],\
           distance.canberra(sent_emd_l1,sent_emd_l2)

df[['l2_dist','l1_dist','cos_dist','cbr_dist']] = df.apply(get_distances, axis=1,result_type='expand')

In [19]:
df.head(1)

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,l1_no_oov_str,l2_no_oov_str,tfidf_l1,tfidf_l2,sent_emd_l1,sent_emd_l2,l2_dist,l1_dist,cos_dist,cbr_dist
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,怎么 更改 手机号码,我 的 是 以前 的 手机号码 怎么 更 改成 现在 的 支付宝 的 号码 手机号,"[0.2911125346008847, 0.6813844622784201, 0.671...","[0.0, 0.0, 0.0, 0.3720269035410146, 0.0, 0.402...","[-0.4842908885329962, 0.6463035643100739, -0.4...","[-0.5333490184275433, 1.1786049455404282, -0.6...",5.10739,69.538723,0.229634,152.813882


In [20]:
df.shape

(102477, 43)

In [21]:

df[['s1', 's2', 'label', 'len_s1', 'len_s2', 'len_s1s2_avg', 'len_s1_on_s2',
       'len_diff', 'edit_dst', 'edit_dst_nzd',
       'len_l1_tkd', 'len_l2_tkd', 'common_wc', 's1_uq_wc', 's2_uq_wc',
       's1_ask', 's2_ask', 'len_l1l2_tkd_avg', 'common_wc_nzd', 's1_uq_wc_nzd',
       's2_uq_wc_nzd', 'bleu1', 'bleu2', 'bleu3', 'bleu4',
       'l2_dist', 'l1_dist', 'cos_dist','cbr_dist']].to_csv('../data/embedded.csv', index=False)


In [22]:
df[['s1', 's2', 'label','l1_no_oov', 'l2_no_oov', 'l1_idx', 'l2_idx']].to_csv('../data/for_nn.csv', index=False)